In [2]:
import pandas as pd
import json
import random
from shapely.geometry import shape, Point

In [3]:
df = pd.read_csv('data/gracy_3-3.csv', encoding='latin1')
df.columns = ['name', 'government', 'county', 'irc', 'iecc', 'notes', 'website', 'source', 'map', 'lat-long', 'column-1']
df[['latitude', 'longitude']] = df['lat-long'].str.extract(r'([0-9.]+)ø[N|S]\s([0-9.]+)ø[E|W]')
# drop where latitute or longitude is missing
df = df.dropna(subset=['latitude', 'longitude'])
df

,name,government,county,irc,iecc,notes,website,source,map,lat-long,column-1,latitude,longitude
0,Aguilar,Statutory town,Las Animas,NaN,NaN,NaN,Town of Aguilar,NaN,map,37.4028øN 104.6533øW,NaN,37.4028,104.6533
1,Akron?[g],Statutory town,Washington,2018.0,2018.0,NaN,Town of Akron,https://townofakron.colorado.gov/building,map,40.1605øN 103.2144øW,NaN,40.1605,103.2144
2,Alamosa?[h],Home rule city,Alamosa,2018.0,2018.0,NaN,City of Alamosa,https://cityofalamosa.org/building-dept/,map,37.4694øN 105.8700øW,NaN,37.4694,105.8700
3,Alma,Statutory town[i],Park,2018.0,2018.0,NaN,Town of Alma,NaN,map,39.2839øN 106.0628øW,NaN,39.2839,106.0628
4,Antonito,Statutory town,Conejos,NaN,NaN,NaN,Town of Antonito,NaN,map,37.0792øN 106.0086øW,NaN,37.0792,106.0086
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,Windsor,Home rule town,"Weld,\nLarimer",2018.0,2018.0,NaN,Town of Windsor,https://library.municode.com/co/windsor/codes/...,map,40.4775øN 104.9014øW,NaN,40.4775,104.9014
268,Winter Park,Home rule town,Grand,NaN,NaN,NaN,Town of Winter Park,NaN,map,39.8917øN 105.7631øW,NaN,39.8917,105.7631
269,Woodland Park,Home rule city,Teller,NaN,NaN,2023 Pikes Peak Regional Building Code,City of Woodland Park,https://library.municode.com/co/woodland_park/...,map,38.9939øN 105.0569øW,NaN,38.9939,105.0569
270,Wray?[dv],Home rule city,Yuma,2018.0,2018.0,NaN,City of Wray,https://library.municode.com/co/wray/codes/cod...,map,40.0758øN 102.2232øW,NaN,40.0758,102.2232


In [5]:


with open('data/geojson.json', 'r') as f:
    geojson = json.load(f)

    # Parse the GeoJSON data into a dictionary where keys are place names and values are geometries
    geojson_dict = {}
    for feature in geojson['features']:
        if 'properties' in feature and 'NAME' in feature['properties'] and 'geometry' in feature:
            geojson_dict[feature['properties']['NAME']] = feature['geometry']

    # Print the number of places found in the GeoJSON
    print(f"Found {len(geojson_dict)} places in the GeoJSON file")

    # Optional: Preview a few entries
    sample_keys = random.sample(list(geojson_dict.keys()), min(5, len(geojson_dict)))
    for key in sample_keys:
        print(f"{key}: {type(geojson_dict[key])}")

    # Convert GeoJSON geometries to Shapely geometries for efficient point-in-polygon queries
    shapely_geometries = {}
    for place_name, geometry in geojson_dict.items():
        try:
            shapely_geometries[place_name] = shape(geometry)
        except Exception as e:
            print(f"Error converting geometry for {place_name}: {e}")

# Function to check if a point is inside any polygon
def find_containing_place(lat, lon):
    point = Point(float(lon), float(lat))  # Note: GeoJSON uses (longitude, latitude) order
    containing_places = []
    
    for place_name, polygon in shapely_geometries.items():
        if polygon.contains(point):
            containing_places.append(place_name)
    
    return containing_places if containing_places else None

# Test with a few points from the dataframe
test_rows = df.iloc[:5]
for i, row in test_rows.iterrows():
    lat, lon = float(row['latitude']), float(row['longitude'])
    places = find_containing_place(lat, lon)
    print(f"{row['name']}: Located in {places}")

Found 0 places in the GeoJSON file
Aguilar: Located in None
Akron?[g]: Located in None
Alamosa?[h]: Located in None
Alma: Located in None
Antonito: Located in None


In [11]:
# Create a GeoJSON file from the dataframe
geojson_points = {
    "type": "FeatureCollection",
    "features": []
}

# Convert each row in the dataframe to a GeoJSON point feature
for idx, row in df.iterrows():
    try:
        lat = float(row['latitude'])
        lon = float(row['longitude'])
        
        # Create feature with properties
        feature = {
            "type": "Feature",
            "geometry": {
            "type": "Point",
            "coordinates": [lon, lat]  # GeoJSON uses [longitude, latitude] order
            },
            "properties": {
            "name": row['name'],
            "government": row['government'],
            "county": row['county'],
            "irc": int(row['irc']) if pd.notna(row['irc']) else "Unknown",
            "iecc": int(row['iecc']) if pd.notna(row['iecc']) else "Unknown",
            "website": row['source']
            }
        }
        
        geojson_points["features"].append(feature)
    except (ValueError, TypeError) as e:
        print(f"Error with row {idx}: {e}")

# Save the GeoJSON file
with open('data/gracy_3-3.geojson', 'w') as f:
    json.dump(geojson_points, f, indent=4)

print(f"Created GeoJSON file with {len(geojson_points['features'])} points")

Created GeoJSON file with 272 points
